In [1]:
! git clone https://github.com/hyintell/BLOOM-fine-tuning.git
%cd BLOOM-fine-tuning
! pip install -r requirements.txt 

c:\Users\Usuario\Downloads\BLOOM-fine-tuning


Cloning into 'BLOOM-fine-tuning'...


     ---------------------------------------- 6.7/6.7 MB 7.2 MB/s eta 0:00:00
     ------------------------------------- 469.0/469.0 kB 28.7 MB/s eta 0:00:00
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
     ---------------------------------------- 224.5/224.5 kB ? eta 0:00:00
  Using cached tokenizers-0.13.3-cp310-cp310-win_amd64.whl (3.5 MB)
     --------------------------------------- 21.5/21.5 MB 27.3 MB/s eta 0:00:00
     ---------------------------------------- 110.5/110.5 kB ? eta 0:00:00
     ---------------------------------------- 134.3/134.3 kB ? eta 0:00:00
     ------------------------------------- 160.1/160.1 kB 10.0 MB/s eta 0:00:00
  Using cached aiohttp-3.8.4-cp310-cp310-win_amd64.whl (319 kB)
  Using cached multidict-6.0.4-cp310-cp310-win_amd64.whl (28 kB)
     ---------------------------------------- 61.0/61.0 kB 3.4 MB/s eta 0:00:00
  Using cached frozenlist-1.3.3-cp310-cp310-win_amd64.whl (33 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Att

# Libraries

In [1]:
import torch
import transformers
from transformers import BloomTokenizerFast, BloomForCausalLM, TrainingArguments

from datasets import load_dataset

from utils import ModifiedTrainer, tokenise_data, data_collator

c:\Users\Usuario\Desktop\Dev\PoLLiBLOOM\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 # Main

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cpu')

In [3]:
model_name = "bloom-1b7"
model = BloomForCausalLM.from_pretrained(f"bigscience/{model_name}")
tokeniser = BloomTokenizerFast.from_pretrained(f"bigscience/{model_name}", add_prefix_space=True)

c:\Users\Usuario\Desktop\Dev\PoLLiBLOOM\.venv\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Usuario\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [4]:
dataset = load_dataset('tatsu-lab/alpaca')
input_ids = tokenise_data(dataset, tokeniser)

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 499.32it/s]


Dataset parquet downloaded and prepared to C:/Users/Usuario/.cache/huggingface/datasets/tatsu-lab___parquet/tatsu-lab--alpaca-715f206eec35a791/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


100%|██████████| 52002/52002 [00:31<00:00, 1673.39it/s]


In [6]:
model.gradient_checkpointing_enable()
model.is_parallelizable = True
model.model_parallel = True

training_args = TrainingArguments(
    "output",
    fp16=False,
    gradient_accumulation_steps= 1,
    per_device_train_batch_size = 2,
    learning_rate = 2e-5,
    num_train_epochs=2,
    logging_steps=10,
    torch_compile=True
)

trainer = ModifiedTrainer(
    model=model,
    train_dataset=input_ids,
    args=training_args,
    data_collator=data_collator,
)

trainer.train()

The speedups for torchdynamo mostly come wih GPU Ampere or higher and which is not detected here.
c:\Users\Usuario\Desktop\Dev\PoLLiBLOOM\.venv\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


RuntimeError: Windows not yet supported for torch.compile